In [2]:
# MAKE SURE FUSION.compute_weights.R IS IN YOUR PATH
# FILL IN THESE PATHS
GCTA="/cellar/users/jlz014/gcta_1.91.4beta/gcta64"
PLINK="<PATH TO PLINK>"
GEMMA="/cellar/users/jlz014/gemma/bin/gemma"
# ALTERNATIVELY: ENSURE THAT plink, gcta, gemma CAN BE CALLED FROM PATH AND REMOVE --PATH_* FLAGS BELOW
# PATH TO DIRECTORY CONTAINING LDREF DATA (FROM FUSION WEBSITE or https://data.broadinstitute.org/alkesgroup/FUSION/LDREF.tar.bz2)
LDREF="/cellar/users/jlz014/fusion_twas-master/LDREF"
# THIS IS USED TO RESTRICT INPUT SNPS TO REFERENCE IDS ONLY

In [3]:
# PATH TO GEUVADIS GENE EXPRESSION MATRIX:
PRE_GEXP="GD462.GeneQuantRPKM.50FN.samplename.resk10.txt"
# GEUVADIS DATA WAS DOWNLOADED FROM https://www.ebi.ac.uk/arrayexpress/experiments/E-GEUV-1/files/analysis_results/

# PATH TO PREFIX FOR GEUVADIS GENOTYPES SPLIT BY CHROMOSOME
# SUBSAMPLE THESE TO THE LDREF SNPS FOR EFFICIENCY
PRE_GENO="1000G"

# PATH TO OUTPUT DIRECTORY (population-specific subdirs will be made)
OUT_DIR="./WEIGHTS"

# ROWS IN THE MATRIX TO ANALYZE (FOR BATCHED RUNS)
BATCH_START=1
BATCH_END=10

In [4]:
# --- BEGIN SCRIPT:

NR="${BATCH_START}_${BATCH_END}"
mkdir --parents tmp/$NR
mkdir --parents hsq/$NR
mkdir --parents out/$NR
# THIS IS DIRECTORY WHERE THE OUTPUT WILL GO:
# mkdir $OUT_DIR

In [5]:
# test first gene in $PRE_GEXP
PARAM=`awk 'NR==7' $PRE_GEXP`

# # Loop through each gene expression phenotype in the batch
# cat $PRE_GEXP | awk -vs=$BATCH_START -ve=$BATCH_END 'NR > s && NR <= e' |  while read PARAM; do

# Get the gene positions +/- 500kb (1MB window)
CHR=`echo $PARAM | awk '{ print $3 }'`
P0=`echo $PARAM | awk '{ print $4 - 0.5e6 }'`
P1=`echo $PARAM | awk '{ print $4 + 0.5e6 }'`
GNAME=`echo $PARAM | awk '{ print $1 }'`

OUT="tmp/$NR/$PRE_GEXP.$GNAME"
echo $GNAME $CHR $P0 $P1

# Pull out the current gene expression phenotype
echo $PARAM | tr ' ' '\n' | tail -n+5 | paste $PRE_GEXP.ID - > $OUT.pheno

# Get the locus genotypes for all samples and set current gene expression as the phenotype
plink2 --bfile $LDREF/$PRE_GENO.EUR.$CHR --pheno $OUT.pheno --make-bed --out $OUT --keep $OUT.pheno --chr $CHR --from-bp $P0 --to-bp $P1 --extract $LDREF/1000G.EUR.$CHR.bim

echo 'Process all samples together (for reference purposes only since this is multi-ethnic data)'
# Process all samples together (for reference purposes only since this is multi-ethnic data)
mkdir $OUT_DIR/ALL
FINAL_OUT="$OUT_DIR/ALL/ALL.$GNAME"
echo $FINAL_OUT

Rscript FUSION.compute_weights.R --bfile $OUT --tmp $OUT.ALL.tmp --out $FINAL_OUT --verbose 0 --save_hsq --PATH_gcta $GCTA --PATH_gemma $GEMMA --models blup,lasso,top1,enet

# ALTERNATIVELY ADD COVARIATES HERE USING THE --covar FLAG
# MINIMAL COMMAND IS: `Rscript FUSION.compute_weights.R --bfile $OUT --tmp $OUT.$pop.tmp --out $FINAL_OUT`

echo 'Append heritability output to hsq file'
# Append heritability output to hsq file
cat $FINAL_OUT.hsq >> hsq/$NR.hsq

# echo 'clean up'
# # Clean-up just in case
# rm -f $FINAL_OUT.hsq $OUT.tmp.*

# echo 'remove all intermediate files'
# # Remove all intermediate files
# rm $OUT.*

echo 'done'

ENSG00000136237.12 7 21896763 22896763
PLINK v1.90p 64-bit (25 Mar 2016)          https://www.cog-genomics.org/plink2
(C) 2005-2016 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000136237.12.log.
Options in effect:
  --bfile /cellar/users/jlz014/fusion_twas-master/LDREF/1000G.EUR.7
  --chr 7
  --extract /cellar/users/jlz014/fusion_twas-master/LDREF/1000G.EUR.7.bim
  --from-bp 21896763
  --keep tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000136237.12.pheno
  --make-bed
  --out tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000136237.12
  --pheno tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000136237.12.pheno
  --to-bp 22896763

257959 MB RAM detected; reserving 128979 MB for main workspace.
673 variants loaded from .bim file.
489 people (0 males, 0 females, 489 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
tmp/1_10/GD462.GeneQuantRPKM.

In [6]:
# Loop through each gene expression phenotype in the batch
cat $PRE_GEXP | awk -vs=$BATCH_START -ve=$BATCH_END 'NR > s && NR <= e' |  while read PARAM; do

# Get the gene positions +/- 500kb (1MB window)
CHR=`echo $PARAM | awk '{ print $3 }'`
P0=`echo $PARAM | awk '{ print $4 - 0.5e6 }'`
P1=`echo $PARAM | awk '{ print $4 + 0.5e6 }'`
GNAME=`echo $PARAM | awk '{ print $1 }'`

OUT="tmp/$NR/$PRE_GEXP.$GNAME"
echo $GNAME $CHR $P0 $P1

# Pull out the current gene expression phenotype
echo $PARAM | tr ' ' '\n' | tail -n+5 | paste $PRE_GEXP.ID - > $OUT.pheno

echo 'Get the locus genotypes for all samples and set current gene expression as the phenotype'

# Get the locus genotypes for all samples and set current gene expression as the phenotype
plink2 --bfile $LDREF/$PRE_GENO.EUR.$CHR --pheno $OUT.pheno --make-bed --out $OUT --keep $OUT.pheno --chr $CHR --from-bp $P0 --to-bp $P1 --extract $LDREF/1000G.EUR.$CHR.bim

echo 'Process all samples together (for reference purposes only since this is multi-ethnic data)'
# Process all samples together (for reference purposes only since this is multi-ethnic data)
mkdir $OUT_DIR/ALL
FINAL_OUT="$OUT_DIR/ALL/ALL.$GNAME"

# remove PATH_* flags because GCTA/GEMMA/PLINK2 can be called from path
Rscript FUSION.compute_weights.R --bfile $OUT --tmp $OUT.tmp --out $FINAL_OUT --verbose 0 --save_hsq --PATH_gcta $GCTA --PATH_gemma $GEMMA --models blup,lasso,top1,enet

# ALTERNATIVELY ADD COVARIATES HERE USING THE --covar FLAG
# MINIMAL COMMAND IS: `Rscript FUSION.compute_weights.R --bfile $OUT --tmp $OUT.$pop.tmp --out $FINAL_OUT`

echo 'Append heritability output to hsq file'
# Append heritability output to hsq file
cat $FINAL_OUT.hsq >> hsq/$NR.hsq

echo 'clean up'
# Clean-up just in case
rm -f $FINAL_OUT.hsq $OUT.tmp.*

echo 'remove all intermediate files'
# Remove all intermediate files
rm $OUT.*

echo 'next gene'
# GO TO THE NEXT GENE
done

ENSG00000152931.6 5 59283540 60283540
Get the locus genotypes for all samples and set current gene expression as the phenotype
PLINK v1.90p 64-bit (25 Mar 2016)          https://www.cog-genomics.org/plink2
(C) 2005-2016 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000152931.6.log.
Options in effect:
  --bfile /cellar/users/jlz014/fusion_twas-master/LDREF/1000G.EUR.5
  --chr 5
  --extract /cellar/users/jlz014/fusion_twas-master/LDREF/1000G.EUR.5.bim
  --from-bp 59283540
  --keep tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000152931.6.pheno
  --make-bed
  --out tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000152931.6
  --pheno tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000152931.6.pheno
  --to-bp 60283540

257959 MB RAM detected; reserving 128979 MB for main workspace.
316 variants loaded from .bim file.
489 people (0 males, 0 females, 489 amb

PLINK v1.90p 64-bit (25 Mar 2016)          https://www.cog-genomics.org/plink2
(C) 2005-2016 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000169129.8.log.
Options in effect:
  --bfile /cellar/users/jlz014/fusion_twas-master/LDREF/1000G.EUR.10
  --chr 10
  --extract /cellar/users/jlz014/fusion_twas-master/LDREF/1000G.EUR.10.bim
  --from-bp 115664515
  --keep tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000169129.8.pheno
  --make-bed
  --out tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000169129.8
  --pheno tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000169129.8.pheno
  --to-bp 116664515

257959 MB RAM detected; reserving 128979 MB for main workspace.
475 variants loaded from .bim file.
489 people (0 males, 0 females, 489 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG0000016

  --chr 15
  --extract /cellar/users/jlz014/fusion_twas-master/LDREF/1000G.EUR.15.bim
  --from-bp 22596869
  --keep tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000259425.1.pheno
  --make-bed
  --out tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000259425.1
  --pheno tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000259425.1.pheno
  --to-bp 23596869

257959 MB RAM detected; reserving 128979 MB for main workspace.
176 variants loaded from .bim file.
489 people (0 males, 0 females, 489 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000259425.1.nosex
.
344 phenotype values present after --pheno.
--extract: 176 variants remaining.
--keep: 344 people remaining.
phenotypes to be ignored, use the --allow-no-sex flag.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 344 founders and 0 nonfounders present.
Calculating allele frequencies... 

In [ ]:
# Process each population
# THIS REQUIRES HAVING A $pop.keep FILE LISTING IDS FOR EACH POPULATION
for pop in EUR YRI TSI CEU FIN GBR; do
	mkdir --parents WEIGHTS/$pop
	FINAL_OUT="WEIGHTS/$pop/$pop.$GNAME"
	# EXTRACT THE POPULATION AND 1% SNPS
	$PLINK --bfile $OUT --keep $pop.keep --make-bed --out $OUT.$pop --maf 0.01

	# MAKE SURE FUSION.compute_weights.R IS IN YOUR PATH
	Rscript FUSION.compute_weights.R --bfile $OUT.$pop --tmp $OUT.$pop.tmp --out $FINAL_OUT --verbose 0 --save_hsq --PATH_gcta $GCTA --PATH_gemma $GEMMA --models blup,lasso,top1,enet
	
	# Append heritability output to hsq file
	cat $FINAL_OUT.hsq >> hsq/$NR.hsq

	# Clean-up just in case
	rm -f $FINAL_OUT.hsq $OUT.tmp.*
done